In [1]:
import pandas as pd
import numpy as np
import random
import gc
import seaborn as sns
import lightgbm as lgb
from matplotlib import pyplot as plt
# import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.metrics import recall_score, auc, accuracy_score, f1_score, precision_score, classification_report, roc_auc_score,mean_squared_error
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime
# import missingno as msno
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_pickle('./data_7d.pickle')
select_features = ['user_id','cp_server_no','cp_role_id','role_created_login_num',
       'role_created_active', 'role_created_online',
       'max_role_level', 'ip_num',
       'device_num',
       'pay_num', 'pay_sum', 'active_0-8', 'active_8-12', 'active_12-14',
       'active_14-18', 'active_18-24', 'pay_grade_1', 'pay_grade_2',
       'pay_grade_3', 'pay_grade_4', 'pay_grade_5', 'pay_grade_6',
       'pay_rate', 'pay_avg',
       'model_money_level', 'hour', 'weekend', 'is_holidays','mobile','platform',
       'user_creates_3_server_num','user_creates_3_role_num', 'time_interval']
label = ['role_created_30_pay_sum']
df = data[select_features].join(data[label])
df['role_created_30_pay_sum'].fillna(0,inplace=True)
df['platform'].fillna(0,inplace=True)
df['user_creates_3_server_num'].fillna(1,inplace=True)
df['user_creates_3_role_num'].fillna(1,inplace=True)
df['time_interval'].fillna(0,inplace=True)
df['model_money_level'].fillna(4,inplace=True)

In [4]:
df['pay_sum'].sum()

42748240.0

In [5]:
df_train, df_test = train_test_split(df,test_size=0.3)
df_train_pay = df_train[df_train['pay_sum']>0]
df_train_nopay = df_train[df_train['pay_sum']==0]

In [6]:
target = df_train_pay['role_created_30_pay_sum']
features = df_train_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
target_ln = np.log1p(target)

In [7]:
X_val, x_test, Y_val, y_test = train_test_split(features,target_ln, test_size=0.3)

In [ ]:
# # lgb的调参与交叉验证
# train_data = lgb.Dataset(X_val, label=Y_val)
# val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

# # 设置参数初始值，不含交叉验证参数
# print('设置参数')
# params = {
#     'boosting_type':'gbdt',
#     'objective':'regression',
#     'metric':'rmse',
#     'nthread':4,
#     'learning_rate':0.05
# }

# # 交叉验证（调参）
# print('交叉验证')
# min_rmse = float('2')
# best_params = {}

# # 准确率
# print('调参1：提高准确率')
# for num_leaves in range(5, 100, 5):
#     for max_depth in range(3, 8, 1):
#         params['num_leaves'] = num_leaves
#         params['max_depth'] = max_depth
        
#         cv_results = lgb.cv(
#                             params,
#                             train_data,
#                             nfold=3,
#                             stratified=False,
#                             early_stopping_rounds=10,
#                             verbose_eval=50     
#                             )
#         mean_rmse = pd.Series(cv_results['rmse-mean']).min()
#         boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
#         if mean_rmse <= min_rmse:
#             min_rmse = mean_rmse
#             best_params['num_leaves'] = num_leaves
#             best_params['max_depth'] = max_depth

# if 'num_leaves' and 'max_depth' in best_params.keys():
#     params['num_leaves'] = best_params['num_leaves']
#     params['max_depth'] = best_params['max_depth']

# # 过拟合
# print('调参2：降低过拟合')
# min_rmse = float('2')
# for max_bin in range(5, 256, 10):
#     for min_data_in_leaf in range(1, 102, 10):
#         params['max_bin'] = max_bin
#         params['min_data_in_leaf'] = min_data_in_leaf
        
#         cv_results = lgb.cv(
#                             params,
#                             train_data,
#                             nfold=3,
#                             stratified=False,
#                             early_stopping_rounds=10,
#                             verbose_eval=50 
#                             )
#         mean_rmse = pd.Series(cv_results['rmse-mean']).min()
#         boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
#         if mean_rmse <= min_rmse:
#             min_rmse = mean_rmse
#             best_params['min_data_in_leaf'] = min_data_in_leaf
#             best_params['max_bin'] = max_bin
            
# if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
#     params['max_bin'] = best_params['max_bin']
#     params['min_data_in_leaf'] = best_params['min_data_in_leaf']

# print('调参3：降低过拟合')
# min_rmse = float('2')
# for feature_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#     for bagging_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#         for bagging_freq in range(0,50,5):
#             params['feature_fraction'] = feature_fraction
#             params['bagging_fraction'] = bagging_fraction
#             params['bagging_freq'] = bagging_freq

#             cv_results = lgb.cv(
#                             params,
#                             train_data,
#                             nfold=3,
#                             stratified=False,
#                             early_stopping_rounds=10,
#                             verbose_eval=50
#                             )
#             mean_rmse = pd.Series(cv_results['rmse-mean']).min()
#             boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
#             if mean_rmse <= min_rmse:
#                 min_rmse = mean_rmse
#                 best_params['feature_fraction'] = feature_fraction
#                 best_params['bagging_fraction'] = bagging_fraction
#                 best_params['bagging_freq'] = bagging_freq

# if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
#     params['feature_fraction'] = best_params['feature_fraction']
#     params['bagging_fraction'] = best_params['bagging_fraction']
#     params['bagging_freq'] = best_params['bagging_freq']

# print('调参4：降低过拟合')
# min_rmse = float('2')
# for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
#     for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
#         params['lambda_l1'] = lambda_l1
#         params['lambda_l2'] = lambda_l2
#         cv_results = lgb.cv(
#                             params,
#                             train_data,
#                             nfold=3,
#                             stratified=False,
#                             early_stopping_rounds=10,
#                             verbose_eval=50
#                             )
#         mean_rmse = pd.Series(cv_results['rmse-mean']).min()
#         boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
#         if mean_rmse <= min_rmse:
#             min_rmse = mean_rmse
#             best_params['lambda_l1'] = lambda_l1
#             best_params['lambda_l2'] = lambda_l2
# if 'lambda_l1' and 'lambda_l2' in best_params.keys():
#     params['lambda_l1'] = best_params['lambda_l1']
#     params['lambda_l2'] = best_params['lambda_l2']

# print("调参5：降低过拟合2")
# min_rmse = float('2')
# for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#     params['min_split_gain'] = min_split_gain
    
#     cv_results = lgb.cv(
#                             params,
#                             train_data,
#                             nfold=3,
#                             stratified=False,
#                             early_stopping_rounds=10,
#                             verbose_eval=50    
#                             )
#     mean_rmse = pd.Series(cv_results['rmse-mean']).min()
#     boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
#     if mean_rmse >= min_rmse:
#         min_rmse = mean_rmse
        
#     best_params['min_split_gain'] = min_split_gain
# if 'min_split_gain' in best_params.keys():
#     params['min_split_gain'] = best_params['min_split_gain']
# print(best_params)

In [8]:
params = {

'boosting_type':'gbdt',
'objective':'regression',
'metric':'rmse',
'nthread':4,
'learning_rate':0.05,
"max_depth":6,
"num_leaves":25,
"max_bin":255,
"min_data_in_leaf":41,
"min_split_gain":1.0,
"feature_fraction": 0.8,
"bagging_fraction":0.9,
"bagging_freq":5,
"lambda_l1":0.3,
"lambda_l2":0.6,
"metric": "rmse",
}
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_r_cv = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

[1]	training's rmse: 1.71698	valid_1's rmse: 1.71359
Training until validation scores don't improve for 100 rounds
[2]	training's rmse: 1.64549	valid_1's rmse: 1.64264
[3]	training's rmse: 1.57812	valid_1's rmse: 1.57574
[4]	training's rmse: 1.51601	valid_1's rmse: 1.51401
[5]	training's rmse: 1.4571	valid_1's rmse: 1.45546
[6]	training's rmse: 1.40099	valid_1's rmse: 1.39986
[7]	training's rmse: 1.34828	valid_1's rmse: 1.34761
[8]	training's rmse: 1.29958	valid_1's rmse: 1.2995
[9]	training's rmse: 1.25397	valid_1's rmse: 1.25437
[10]	training's rmse: 1.21053	valid_1's rmse: 1.21143
[11]	training's rmse: 1.17	valid_1's rmse: 1.17136
[12]	training's rmse: 1.13207	valid_1's rmse: 1.13394
[13]	training's rmse: 1.0973	valid_1's rmse: 1.09967
[14]	training's rmse: 1.06517	valid_1's rmse: 1.06823
[15]	training's rmse: 1.03433	valid_1's rmse: 1.03795
[16]	training's rmse: 1.00572	valid_1's rmse: 1.00981
[17]	training's rmse: 0.979094	valid_1's rmse: 0.983639
[18]	training's rmse: 0.954401	va

[176]	training's rmse: 0.661293	valid_1's rmse: 0.690061
[177]	training's rmse: 0.66129	valid_1's rmse: 0.690059
[178]	training's rmse: 0.661289	valid_1's rmse: 0.690056
[179]	training's rmse: 0.661288	valid_1's rmse: 0.690054
[180]	training's rmse: 0.661287	valid_1's rmse: 0.690052
[181]	training's rmse: 0.661234	valid_1's rmse: 0.690055
[182]	training's rmse: 0.661191	valid_1's rmse: 0.690056
[183]	training's rmse: 0.661145	valid_1's rmse: 0.69005
[184]	training's rmse: 0.661115	valid_1's rmse: 0.690057
[185]	training's rmse: 0.661114	valid_1's rmse: 0.690056
[186]	training's rmse: 0.661112	valid_1's rmse: 0.690055
[187]	training's rmse: 0.661111	valid_1's rmse: 0.690054
[188]	training's rmse: 0.66111	valid_1's rmse: 0.690052
[189]	training's rmse: 0.661105	valid_1's rmse: 0.690048
[190]	training's rmse: 0.661101	valid_1's rmse: 0.690047
[191]	training's rmse: 0.661098	valid_1's rmse: 0.690049
[192]	training's rmse: 0.661012	valid_1's rmse: 0.690088
[193]	training's rmse: 0.66101	val

In [9]:
y_predict_lgb = lgb_r_cv.predict(x_test)
mse = mean_squared_error(np.expm1(y_predict_lgb), np.expm1(y_test))
mae = mean_absolute_error(np.expm1(y_predict_lgb), np.expm1(y_test))
rmse = mse ** 0.5
print(rmse)
print(mse)
print(mae)

2080.316018648801
4327714.7374468
383.18412920895855


In [10]:
import joblib
joblib.dump(lgb_r_cv, './lgb_7d.pkl')

['./lgb_7d.pkl']

In [11]:
# 用df_test数据集进行测试

In [12]:
df_test_pay = df_test[df_test['pay_sum']>0]
df_test_nopay = df_test[df_test['pay_sum']==0]
df_test_part1 = df_test_nopay[['user_id','cp_server_no','cp_role_id','pay_num']].rename(columns=
                                                                                {'pay_num':'predict_30_pay'})

In [13]:
target_test = df_test_pay['role_created_30_pay_sum']
target_test_ln = np.log1p(target_test)
features_test = df_test_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
y_predict = lgb_r_cv.predict(features_test)
y_predict[y_predict<0] = 0
mse = mean_squared_error(np.expm1(y_predict), np.expm1(target_test_ln))
mae = mean_absolute_error(np.expm1(y_predict), np.expm1(target_test_ln))
rmse = mse ** 0.5
print(rmse)
print(mse)
print(mae)

2184.225783145421
4770842.271757226
408.81404716355314


In [14]:
sum(np.expm1(target_test_ln))/sum(np.expm1(y_predict))

1.2623073109285605

In [15]:
df_test_part2 = df_test_pay[['user_id','cp_server_no','cp_role_id']]
df_test_part2['predict_30_pay'] = np.expm1(y_predict)*1.24
pred = df_test_part1.append(df_test_part2)
predict_data = pd.merge(df_test[['user_id','cp_server_no','cp_role_id', 'role_created_30_pay_sum']],pred,on=['user_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [16]:
mse = mean_squared_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
mae = mean_absolute_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
rmse = mse ** 0.5
print('测试集上的均方根误差:%.2f元'% rmse)
# print('测试集上的均方误差:%f元' % mse)
print('测试集上的平均绝对误差:%.2f元'% mae)

测试集上的均方根误差:330.09元
测试集上的平均绝对误差:10.81元


In [17]:
print('测试集前30天实际总的付费金额:%.2f元' % predict_data['role_created_30_pay_sum'].sum())
print('测试集前30天预测总的付费金额:%.2f元'% predict_data['predict_30_pay'].sum())
print('预测总金额准确率:',predict_data['predict_30_pay'].sum()/predict_data['role_created_30_pay_sum'].sum())

测试集前30天实际总的付费金额:30208548.00元
测试集前30天预测总的付费金额:28959106.43元
预测总金额准确率: 0.958639469411031


# 分计划

In [18]:
data['create_role_time'] = data['create_role_time'].dt.date

In [19]:
df_source = data[['user_id','cp_server_no','cp_role_id','create_role_time','channel_id','source_id','pay_sum']]
df_source.drop_duplicates(inplace=True)
df_source_predict = pd.merge(predict_data,df_source,on=['user_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [20]:
df_source_predict.head()

,user_id,cp_server_no,cp_role_id,role_created_30_pay_sum,predict_30_pay,create_role_time,channel_id,source_id,pay_sum
0,104656031,660393,14968896,0.0,0.0,2020-03-03,20541.0,231026.0,0.0
1,112420763,660772,17464094,0.0,0.0,2020-05-04,20545.0,301827.0,0.0
2,107282815,660483,15877917,0.0,0.0,2020-03-16,20953.0,281020.0,0.0
3,109672079,660610,16653456,0.0,0.0,2020-04-03,20636.0,287062.0,0.0
4,101563639,660239,14120243,0.0,0.0,2020-01-27,6770.0,260200.0,0.0


In [21]:
df_source_predict[(df_source_predict['channel_id']==7162)&(df_source_predict['source_id']==226304)]

,user_id,cp_server_no,cp_role_id,role_created_30_pay_sum,predict_30_pay,create_role_time,channel_id,source_id,pay_sum
35572,97264636,660094,13375457,0.0,0.000000,2019-11-09,7162.0,226304.0,0.0
72063,97175250,660092,13362709,0.0,0.000000,2019-11-08,7162.0,226304.0,0.0
74406,100253618,660192,13812730,0.0,0.000000,2020-01-02,7162.0,226304.0,0.0
78392,98155331,660111,13473438,0.0,0.000000,2019-11-18,7162.0,226304.0,0.0
81987,97265482,660094,13375554,0.0,0.000000,2019-11-09,7162.0,226304.0,0.0
98965,98873285,660136,13585330,0.0,0.000000,2019-12-01,7162.0,226304.0,0.0
153170,98495502,660123,13528861,0.0,0.000000,2019-11-24,7162.0,226304.0,0.0
159268,98833733,660135,13580506,0.0,0.000000,2019-11-30,7162.0,226304.0,0.0
229398,97220756,660093,13370115,0.0,0.000000,2019-11-09,7162.0,226304.0,0.0
236505,97262362,660094,13375174,0.0,0.000000,2019-11-09,7162.0,226304.0,0.0


In [68]:
df_source_predict['is_pay'] = (df_source_predict['pay_sum'] != 0).astype(int)
groups = df_source_predict.groupby(['channel_id','source_id','create_role_time'])
print(groups.ngroups)
temp1 = groups['role_created_30_pay_sum'].agg([('30_pay_sum','sum')]).reset_index()
temp2 = groups['predict_30_pay'].agg([('predict_30_pay','sum')]).reset_index()
temp3 = groups['pay_sum'].agg([('n_pay_sum','sum')]).reset_index()
temp4 = groups['user_id'].agg([('n_user_sum','count')]).reset_index()
temp5 = groups['is_pay'].agg([('pay_user_sum','sum')]).reset_index()

125515


In [69]:
df_source_predict = pd.merge(temp1,temp2,on=['channel_id','source_id','create_role_time'],how='inner',validate='one_to_one')
df_source_predict = pd.merge(df_source_predict,temp3,on=['channel_id','source_id','create_role_time'],how='inner',validate='one_to_one')
df_source_predict = pd.merge(df_source_predict,temp4,on=['channel_id','source_id','create_role_time'],how='inner',validate='one_to_one')
df_source_predict = pd.merge(df_source_predict,temp5,on=['channel_id','source_id','create_role_time'],how='inner',validate='one_to_one')

In [70]:
def weight_error(df):
    df['error'] = np.abs((df['predict_30_pay']-df['30_pay_sum'])/df['30_pay_sum'])
    df['weight'] = df['30_pay_sum'] / df['30_pay_sum'].sum()
    df['weight_error'] = df_source_predict['weight'] * df_source_predict['error']
    return df['weight_error'].sum()

In [71]:
weight_error(df_source_predict)

0.47079341703787614

In [72]:
print(weight_error(df_source_predict[df_source_predict['n_user_sum']>50]))
print(df_source_predict[df_source_predict['n_user_sum']>4].shape[0])

0.24427928006803526
32908


In [73]:
print(weight_error(df_source_predict[df_source_predict['pay_user_sum']>5]))
print(df_source_predict[df_source_predict['pay_user_sum']>5].shape[0])

0.10907878520792673
1043


In [74]:
print(weight_error(df_source_predict[df_source_predict['pay_user_sum']>3]))
print(df_source_predict[df_source_predict['pay_user_sum']>3].shape[0])

0.1679130535165111
2014


In [75]:
print(weight_error(df_source_predict[df_source_predict['pay_user_sum']>2]))
print(df_source_predict[df_source_predict['pay_user_sum']>2].shape[0])

0.21819746842823634
3170
